In [21]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import math
import time
from pathlib import Path


class FoodTriplets(Dataset):
    """
    Class to load food triplets. Individual items consist of:
     - x: concatenated image features of a triplet (ABC)
     - y: corresponding label (1: B is more similar, 0: C is more similar, None: no label)
    """
    def __init__(self, features_file, triplets_file, is_labelled_data=False):
        print("initializing " + str(triplets_file) + " dataset...")
        self.img_features = pd.read_csv("data/" + features_file, header=None, index_col=0)
        self.triplets = pd.read_csv("data/" + triplets_file, sep=" ", header=None).to_numpy()
        self.is_labelled_data = is_labelled_data
        self.labels = None

        if self.is_labelled_data:
            # For each triplet ABC add a triplet ACB
            # Also add corresponding labels
            dim = self.triplets.shape
            extended_triplets = np.zeros((2*dim[0], dim[1]), dtype=np.int32)
            self.labels = np.zeros(2*dim[0], dtype=np.bool)
            idx = 0
            for triplet in self.triplets:
                extended_triplets[idx, :] = triplet
                extended_triplets[idx + 1, :] = [triplet[0], triplet[2], triplet[1]]
                self.labels[idx] = 1  # label at idx+1 is already initialized to 0
                idx += 2
        print("done")

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, idx):
        triplet = self.triplets[idx]
        a, b, c = triplet[0], triplet[1], triplet[2]
        a_features = self.img_features.loc[[a]].to_numpy(dtype=np.float32)
        b_features = self.img_features.loc[[b]].to_numpy(dtype=np.float32)
        c_features = self.img_features.loc[[c]].to_numpy(dtype=np.float32)
        features = np.concatenate((a_features, b_features, c_features), axis=1)
        label = 0  # dummy label
        if self.is_labelled_data:
            label = np.array([[self.labels[idx]]])
        return {"x": features, "y": label}


def get_simple_nn():
    net = nn.Sequential(
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=12288, out_features=6144),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=6144, out_features=3072),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=3072, out_features=1500),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=1500, out_features=500),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=500, out_features=250),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=250, out_features=1)
    )
    return net

# def get_first_try_nn():
#     # input 3x1000
#     # L1    1500
#     # L2     750
#     # L3
#     # OUTPUT  1
#     net = nn.Sequential(
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=12288, out_features=6144),
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=6144, out_features=3072),
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=3072, out_features=1500)
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=1500, out_features=500)
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=500, out_features=250)
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=250, out_features=1)
#     )

#     return net

def timer(start, end):
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds)

 
torch.manual_seed(42)

num_epochs = 100  # each epoch is one pass over the whole dataset
batch_size = 16  # how many samples per training step (32,64,128)
num_workers = 1  # how many CPU cores to use

# Specify some file names
img_features_file = "train_image_features_vgg16bn.csv"
train_triplets_file = "train_triplets.txt" #"train_triplets.txt"
test_triplets_file = "test_triplets.txt" #"test_triplets.txt"

# initialize datasets and dataloaders
train_data = FoodTriplets(img_features_file, train_triplets_file, is_labelled_data=True)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_data = FoodTriplets(img_features_file, test_triplets_file, is_labelled_data=False)
test_dataloader = DataLoader(test_data, batch_size=None, shuffle=False, num_workers=num_workers)


initializing train_triplets.txt dataset...
done
initializing test_triplets.txt dataset...
done


In [22]:
def train_model(model, num_epochs=30):
    
    model.to(device)
    
    loss_function = nn.BCEWithLogitsLoss()  # combines sigmoid with binary cross entropy loss function
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    n_batches = math.ceil(len(train_data) / batch_size)
 
    print(f"training started at {time.ctime()}")
    start_t = time.time()
    print_frequency = 100
    
    for epoch in range(num_epochs):
        running_loss = 0.0
#         print("Epoch {}/{}".format(epoch, num_epochs-1))
 
        for i, data in enumerate(train_dataloader):
            inputs = data["x"].float()
            labels = data["y"].float()
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()   

            output = model(inputs)
            
            loss = loss_function(output, labels)
            loss.backward() # Calculate gradients
            optimizer.step() # Update weights

            # print statistics
            running_loss += loss.item()
            if i % print_frequency == 0:
                loss_history.append(running_loss/print_frequency)
#                 print("%d, %5d, loss: %.3f" % (epoch, i, running_loss/100))
                print(f"Epoch: {epoch+1}/{num_epochs}, Batch: {i}/{n_batches}, loss: {running_loss/print_frequency}, "
                  f"Time elapsed: {timer(start_t, time.time())}")
                running_loss = 0
        
        
     
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

loss_history = []

model = get_simple_nn()

train_model(model=model,num_epochs=num_epochs)

training started at Fri May 21 19:40:40 2021
Epoch: 1/100, Batch: 0/3720, loss: 0.006901883482933044, Time elapsed: 00:00:00.11
Epoch: 1/100, Batch: 100/3720, loss: 23.336764764785766, Time elapsed: 00:00:03.13
Epoch: 1/100, Batch: 200/3720, loss: 23.218885398805142, Time elapsed: 00:00:06.11
Epoch: 1/100, Batch: 300/3720, loss: 6.27902067065239, Time elapsed: 00:00:09.10
Epoch: 1/100, Batch: 400/3720, loss: 7.666870151758194, Time elapsed: 00:00:12.09
Epoch: 1/100, Batch: 500/3720, loss: 2.5085186129808426, Time elapsed: 00:00:15.08
Epoch: 1/100, Batch: 600/3720, loss: 10.177432960271835, Time elapsed: 00:00:18.06
Epoch: 1/100, Batch: 700/3720, loss: 2.615809427499771, Time elapsed: 00:00:21.05
Epoch: 1/100, Batch: 800/3720, loss: 1.7338310301303863, Time elapsed: 00:00:24.03
Epoch: 1/100, Batch: 900/3720, loss: 1.1761975169181824, Time elapsed: 00:00:27.07
Epoch: 1/100, Batch: 1000/3720, loss: 1.767425680756569, Time elapsed: 00:00:30.17
Epoch: 1/100, Batch: 1100/3720, loss: 1.211224

Epoch: 3/100, Batch: 2200/3720, loss: 1.3959294635057449, Time elapsed: 00:04:59.90
Epoch: 3/100, Batch: 2300/3720, loss: 0.7026662659645081, Time elapsed: 00:05:03.10
Epoch: 3/100, Batch: 2400/3720, loss: 1.9965558302402497, Time elapsed: 00:05:06.29
Epoch: 3/100, Batch: 2500/3720, loss: 0.6934102106094361, Time elapsed: 00:05:09.49
Epoch: 3/100, Batch: 2600/3720, loss: 1.1172626149654388, Time elapsed: 00:05:12.67
Epoch: 3/100, Batch: 2700/3720, loss: 0.692616708278656, Time elapsed: 00:05:15.86
Epoch: 3/100, Batch: 2800/3720, loss: 1.3441723555326461, Time elapsed: 00:05:19.07
Epoch: 3/100, Batch: 2900/3720, loss: 0.6945906418561936, Time elapsed: 00:05:22.27
Epoch: 3/100, Batch: 3000/3720, loss: 0.746846210360527, Time elapsed: 00:05:25.45
Epoch: 3/100, Batch: 3100/3720, loss: 0.8840188920497895, Time elapsed: 00:05:28.65
Epoch: 3/100, Batch: 3200/3720, loss: 0.6933779609203339, Time elapsed: 00:05:31.84
Epoch: 3/100, Batch: 3300/3720, loss: 1.387117400765419, Time elapsed: 00:05:3

Epoch: 6/100, Batch: 700/3720, loss: 0.6936332261562348, Time elapsed: 00:10:08.38
Epoch: 6/100, Batch: 800/3720, loss: 0.7192876809835433, Time elapsed: 00:10:11.56
Epoch: 6/100, Batch: 900/3720, loss: 0.693713201880455, Time elapsed: 00:10:14.75
Epoch: 6/100, Batch: 1000/3720, loss: 0.9674060207605362, Time elapsed: 00:10:17.94
Epoch: 6/100, Batch: 1100/3720, loss: 0.6942847448587418, Time elapsed: 00:10:21.13
Epoch: 6/100, Batch: 1200/3720, loss: 0.6933063232898712, Time elapsed: 00:10:24.32
Epoch: 6/100, Batch: 1300/3720, loss: 0.8946020722389221, Time elapsed: 00:10:27.50
Epoch: 6/100, Batch: 1400/3720, loss: 0.6933845430612564, Time elapsed: 00:10:30.70
Epoch: 6/100, Batch: 1500/3720, loss: 0.6936496889591217, Time elapsed: 00:10:33.89
Epoch: 6/100, Batch: 1600/3720, loss: 0.6935088312625886, Time elapsed: 00:10:37.07
Epoch: 6/100, Batch: 1700/3720, loss: 0.6931456661224366, Time elapsed: 00:10:40.27
Epoch: 6/100, Batch: 1800/3720, loss: 0.6927388644218445, Time elapsed: 00:10:43

Epoch: 8/100, Batch: 2900/3720, loss: 0.6935070657730102, Time elapsed: 00:15:15.81
Epoch: 8/100, Batch: 3000/3720, loss: 0.6929809147119522, Time elapsed: 00:15:19.01
Epoch: 8/100, Batch: 3100/3720, loss: 0.6938405680656433, Time elapsed: 00:15:22.20
Epoch: 8/100, Batch: 3200/3720, loss: 0.693503959774971, Time elapsed: 00:15:25.39
Epoch: 8/100, Batch: 3300/3720, loss: 0.6938576865196228, Time elapsed: 00:15:28.57
Epoch: 8/100, Batch: 3400/3720, loss: 0.6934014111757278, Time elapsed: 00:15:31.77
Epoch: 8/100, Batch: 3500/3720, loss: 0.7306930309534073, Time elapsed: 00:15:34.95
Epoch: 8/100, Batch: 3600/3720, loss: 0.6929251450300217, Time elapsed: 00:15:38.14
Epoch: 8/100, Batch: 3700/3720, loss: 0.6938660424947739, Time elapsed: 00:15:41.31
Epoch: 9/100, Batch: 0/3720, loss: 0.0069984769821167, Time elapsed: 00:15:42.05
Epoch: 9/100, Batch: 100/3720, loss: 0.6935827541351318, Time elapsed: 00:15:45.23
Epoch: 9/100, Batch: 200/3720, loss: 0.693285807967186, Time elapsed: 00:15:48.40

Epoch: 11/100, Batch: 1300/3720, loss: 1.8728184604644775, Time elapsed: 00:20:20.81
Epoch: 11/100, Batch: 1400/3720, loss: 0.6937531930208206, Time elapsed: 00:20:24.00
Epoch: 11/100, Batch: 1500/3720, loss: 0.6929265952110291, Time elapsed: 00:20:27.19
Epoch: 11/100, Batch: 1600/3720, loss: 0.6932508456707, Time elapsed: 00:20:30.39
Epoch: 11/100, Batch: 1700/3720, loss: 0.6934246957302094, Time elapsed: 00:20:33.57
Epoch: 11/100, Batch: 1800/3720, loss: 0.6922467023134231, Time elapsed: 00:20:36.77
Epoch: 11/100, Batch: 1900/3720, loss: 0.6936699515581131, Time elapsed: 00:20:39.97
Epoch: 11/100, Batch: 2000/3720, loss: 0.6927467656135559, Time elapsed: 00:20:43.16
Epoch: 11/100, Batch: 2100/3720, loss: 0.6937501436471939, Time elapsed: 00:20:46.34
Epoch: 11/100, Batch: 2200/3720, loss: 0.6928353726863861, Time elapsed: 00:20:49.52
Epoch: 11/100, Batch: 2300/3720, loss: 0.6919309371709823, Time elapsed: 00:20:52.72
Epoch: 11/100, Batch: 2400/3720, loss: 0.6939107811450959, Time elap

...

Epoch: 90/100, Batch: 1800/3720, loss: 0.6939312732219696, Time elapsed: 02:53:28.84
Epoch: 90/100, Batch: 1900/3720, loss: 0.6938090628385544, Time elapsed: 02:53:31.83
Epoch: 90/100, Batch: 2000/3720, loss: 0.6929792809486389, Time elapsed: 02:53:34.82
Epoch: 90/100, Batch: 2100/3720, loss: 0.6937679886817932, Time elapsed: 02:53:37.89
Epoch: 90/100, Batch: 2200/3720, loss: 0.6934945797920227, Time elapsed: 02:53:40.90
Epoch: 90/100, Batch: 2300/3720, loss: 0.693722795844078, Time elapsed: 02:53:43.91
Epoch: 90/100, Batch: 2400/3720, loss: 0.6935042291879654, Time elapsed: 02:53:46.92
Epoch: 90/100, Batch: 2500/3720, loss: 0.6934545201063156, Time elapsed: 02:53:49.94
Epoch: 90/100, Batch: 2600/3720, loss: 0.6933192205429077, Time elapsed: 02:53:52.98
Epoch: 90/100, Batch: 2700/3720, loss: 0.693232154250145, Time elapsed: 02:53:56.08
Epoch: 90/100, Batch: 2800/3720, loss: 0.6938597965240478, Time elapsed: 02:53:59.23
Epoch: 90/100, Batch: 2900/3720, loss: 0.6934387159347534, Time ela

Epoch: 93/100, Batch: 100/3720, loss: 0.6933339244127273, Time elapsed: 02:58:26.92
Epoch: 93/100, Batch: 200/3720, loss: 0.6939514017105103, Time elapsed: 02:58:30.07
Epoch: 93/100, Batch: 300/3720, loss: 0.6924386560916901, Time elapsed: 02:58:33.33
Epoch: 93/100, Batch: 400/3720, loss: 0.6929962807893753, Time elapsed: 02:58:36.43
Epoch: 93/100, Batch: 500/3720, loss: 0.6937062293291092, Time elapsed: 02:58:39.55
Epoch: 93/100, Batch: 600/3720, loss: 0.6936805623769761, Time elapsed: 02:58:42.59
Epoch: 93/100, Batch: 700/3720, loss: 0.6927872425317765, Time elapsed: 02:58:45.76
Epoch: 93/100, Batch: 800/3720, loss: 0.6936782199144363, Time elapsed: 02:58:49.05
Epoch: 93/100, Batch: 900/3720, loss: 0.6927959215641022, Time elapsed: 02:58:52.11
Epoch: 93/100, Batch: 1000/3720, loss: 0.6927073812484741, Time elapsed: 02:58:55.15
Epoch: 93/100, Batch: 1100/3720, loss: 0.6937299174070358, Time elapsed: 02:58:58.46
Epoch: 93/100, Batch: 1200/3720, loss: 0.6927133685350418, Time elapsed: 0

Epoch: 95/100, Batch: 2200/3720, loss: 0.6937492799758911, Time elapsed: 03:03:20.57
Epoch: 95/100, Batch: 2300/3720, loss: 0.6932363945245743, Time elapsed: 03:03:23.65
Epoch: 95/100, Batch: 2400/3720, loss: 0.6927396386861802, Time elapsed: 03:03:26.68
Epoch: 95/100, Batch: 2500/3720, loss: 0.6935667926073075, Time elapsed: 03:03:29.71
Epoch: 95/100, Batch: 2600/3720, loss: 0.6939850050210953, Time elapsed: 03:03:32.73
Epoch: 95/100, Batch: 2700/3720, loss: 0.6933987194299698, Time elapsed: 03:03:35.76
Epoch: 95/100, Batch: 2800/3720, loss: 0.6939174491167068, Time elapsed: 03:03:38.79
Epoch: 95/100, Batch: 2900/3720, loss: 0.6933337658643722, Time elapsed: 03:03:41.82
Epoch: 95/100, Batch: 3000/3720, loss: 0.6935333877801895, Time elapsed: 03:03:44.86
Epoch: 95/100, Batch: 3100/3720, loss: 0.6930654489994049, Time elapsed: 03:03:47.89
Epoch: 95/100, Batch: 3200/3720, loss: 0.6933102691173554, Time elapsed: 03:03:50.92
Epoch: 95/100, Batch: 3300/3720, loss: 0.6932649517059326, Time e

Epoch: 98/100, Batch: 500/3720, loss: 0.6945061153173446, Time elapsed: 03:08:10.51
Epoch: 98/100, Batch: 600/3720, loss: 0.6929041749238968, Time elapsed: 03:08:13.57
Epoch: 98/100, Batch: 700/3720, loss: 0.6929036164283753, Time elapsed: 03:08:16.88
Epoch: 98/100, Batch: 800/3720, loss: 0.6940332198143006, Time elapsed: 03:08:20.00
Epoch: 98/100, Batch: 900/3720, loss: 0.6934933459758759, Time elapsed: 03:08:23.05
Epoch: 98/100, Batch: 1000/3720, loss: 0.6934694600105286, Time elapsed: 03:08:26.06
Epoch: 98/100, Batch: 1100/3720, loss: 0.6937363004684448, Time elapsed: 03:08:29.12
Epoch: 98/100, Batch: 1200/3720, loss: 0.6936226856708526, Time elapsed: 03:08:32.25
Epoch: 98/100, Batch: 1300/3720, loss: 0.6939501762390137, Time elapsed: 03:08:35.53
Epoch: 98/100, Batch: 1400/3720, loss: 0.6934710103273392, Time elapsed: 03:08:38.55
Epoch: 98/100, Batch: 1500/3720, loss: 0.6934943503141403, Time elapsed: 03:08:41.58
Epoch: 98/100, Batch: 1600/3720, loss: 0.6935083800554276, Time elapse

Epoch: 100/100, Batch: 2600/3720, loss: 0.6929003125429154, Time elapsed: 03:13:12.84
Epoch: 100/100, Batch: 2700/3720, loss: 0.6929036897420883, Time elapsed: 03:13:16.22
Epoch: 100/100, Batch: 2800/3720, loss: 0.6925165545940399, Time elapsed: 03:13:19.46
Epoch: 100/100, Batch: 2900/3720, loss: 0.6934862983226776, Time elapsed: 03:13:22.55
Epoch: 100/100, Batch: 3000/3720, loss: 0.6921421456336975, Time elapsed: 03:13:25.59
Epoch: 100/100, Batch: 3100/3720, loss: 0.6941951769590378, Time elapsed: 03:13:28.57
Epoch: 100/100, Batch: 3200/3720, loss: 0.6938539558649063, Time elapsed: 03:13:31.56
Epoch: 100/100, Batch: 3300/3720, loss: 0.6933326846361161, Time elapsed: 03:13:34.55
Epoch: 100/100, Batch: 3400/3720, loss: 0.6933960962295532, Time elapsed: 03:13:37.54
Epoch: 100/100, Batch: 3500/3720, loss: 0.6935535389184951, Time elapsed: 03:13:40.76
Epoch: 100/100, Batch: 3600/3720, loss: 0.6935989981889725, Time elapsed: 03:13:44.01
Epoch: 100/100, Batch: 3700/3720, loss: 0.693594016432

In [24]:
model = model.to('cpu')

# predict on test_data
model_pred = nn.Sequential(model, nn.Softmax(dim=0))  # add softmax to get 1/0 output
predictions = np.zeros(len(test_data))
print("predicting on test_data...")
l_test = len(test_data)
start_t = time.time()

model_pred.to(device)

for i, data in enumerate(test_dataloader, 0):
    x = data["x"].float()
    
    x = x.to(device)
    predictions[i] = model_pred(x)
#     predictions[i] = model(x)
    if i % 1000 == 0:
        print(f"Predicted: {i}/{l_test}, Time elapsed: {timer(start_t, time.time())}")
        
        
# predictions_INT = pd.DataFrame(predictions).astype(int)
predictions_DF = pd.DataFrame(predictions)
predictions_rounded = predictions_DF.round(0)

# print("saving predictions...")
# predictions_INT.to_csv('data/predictions.csv', index=False, header=False)
print("done")

predicting on test_data...
Predicted: 0/59544, Time elapsed: 00:00:00.19
Predicted: 1000/59544, Time elapsed: 00:00:01.82
Predicted: 2000/59544, Time elapsed: 00:00:03.44
Predicted: 3000/59544, Time elapsed: 00:00:05.05
Predicted: 4000/59544, Time elapsed: 00:00:06.65
Predicted: 5000/59544, Time elapsed: 00:00:08.28
Predicted: 6000/59544, Time elapsed: 00:00:09.89
Predicted: 7000/59544, Time elapsed: 00:00:11.50
Predicted: 8000/59544, Time elapsed: 00:00:13.15
Predicted: 9000/59544, Time elapsed: 00:00:14.92
Predicted: 10000/59544, Time elapsed: 00:00:16.62
Predicted: 11000/59544, Time elapsed: 00:00:18.23
Predicted: 12000/59544, Time elapsed: 00:00:19.86
Predicted: 13000/59544, Time elapsed: 00:00:21.47
Predicted: 14000/59544, Time elapsed: 00:00:23.10
Predicted: 15000/59544, Time elapsed: 00:00:24.71
Predicted: 16000/59544, Time elapsed: 00:00:26.33
Predicted: 17000/59544, Time elapsed: 00:00:28.05
Predicted: 18000/59544, Time elapsed: 00:00:29.68
Predicted: 19000/59544, Time elapsed

In [25]:
# res = pd.DataFrame(predictions_rounded).astype(int)
res = pd.DataFrame(predictions_rounded)
print("saving predictions...")
res.to_csv('data/predictions_sigmoid.csv', index=False, header=False)
print("done")

res.head(100)

saving predictions...
done


,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
95,1.0
96,1.0
97,1.0
98,1.0


In [23]:
Path("model").mkdir(parents=True, exist_ok=True)  # create folder if necessary
torch.save(model.state_dict(), "model/params_6_layers")
# load: model.load_state_dict(torch.load("model/params"))
